In [2]:
# Translated to .py by Yundi Zhang
# Jan 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:00:22 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable

df = pd.read_csv("netherlands.dat",'\t')
database = db.Database("netherlands",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

exclude = sp != 0
database.remove(exclude)

# Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR		 = Beta('ASC_CAR',0,None,None,0)
ASC_RAIL	 = Beta('ASC_RAIL',0,None,None,1)
BETA_COST	 = Beta('BETA_COST',0,None,None,0)
BETA_TT_CAR	 = Beta('BETA_TT_CAR',0,None,None,0)
BETA_TT_RAIL = Beta('BETA_TT_RAIL',0,None,None,0)

# Define here arithmetic expressions for name that are not directly available from the data
rail_time  = DefineVariable('rail_time',(  rail_ivtt   +  rail_acc_time   ) +  rail_egr_time  ,database)
car_time  = DefineVariable('car_time', car_ivtt   +  car_walk_time  ,database)
rate_G2E = DefineVariable('rate_G2E', 0.44378022,database)
car_cost_euro = DefineVariable('car_cost_euro', car_cost * rate_G2E,database)
rail_cost_euro = DefineVariable('rail_cost_euro', rail_cost * rate_G2E,database)

# Utilities
__Car = ASC_CAR  + BETA_COST * car_cost_euro + BETA_TT_CAR * car_time
__Rail = ASC_RAIL + BETA_COST * rail_cost_euro + BETA_TT_RAIL * rail_time
__V = {0: __Car,1: __Rail}
__av = {0: 1,1: 1}

# The choice model is a logit, with availability conditions
logprob = models.loglogit(__V,__av,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "binary_specific_netherlands"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


              Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_CAR        2.43     1.06     2.3   0.0216         0.973          2.5   
BETA_COST    -0.123   0.0243   -5.04  4.6e-07        0.0256        -4.79   
BETA_TT_CAR   -2.26    0.479   -4.72 2.31e-06         0.485        -4.66   
BETA_TT_RAIL -0.543    0.417    -1.3    0.193         0.396        -1.37   

              Rob. p-value  
ASC_CAR             0.0125  
BETA_COST         1.71e-06  
BETA_TT_CAR       3.14e-06  
BETA_TT_RAIL          0.17  
Nbr of observations: 228
LL(0) =    -158.038
LL(beta) = -118.023
rho bar square = 0.228
Output file: binary_specific_netherlands~00.html
